In [ ]:
import sagemaker
import boto3

sm_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sm_session.boto_region_name
account = sm_session.account_id()


container_name = "deepspeed-sm"
container_tag = "latest"
image = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account, region, container_name, container_tag)

In [ ]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin 763104351884.dkr.ecr.{region}.amazonaws.com
# loging to your private ECR
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account}.dkr.ecr.{region}.amazonaws.com

In [4]:
! ./build_and_push.sh $container_name $container_tag Dockerfile

Working in region us-east-1
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  604.2kB
Step 1/35 : ARG REGION
Step 2/35 : FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu101-ubuntu16.04
 ---> 0d7223a296dd
Step 3/35 : ENV STAGE_DIR=/tmp
 ---> Using cache
 ---> 162c609ace50
Step 4/35 : RUN mkdir -p ${STAGE_DIR}
 ---> Using cache
 ---> f0877b798876
Step 5/35 : RUN apt-get update &&     apt-get install -y --no-install-recommends         software-properties-common autotools-dev         nfs-common pdsh g++ gcc tmux less unzip         htop iftop iotop rsync iputils-ping         net-tools         sudo
 ---> Using cache
 ---> eb3c8042f059
Step 6/35 : RUN wget h

In [ ]:
! docker images ls

In [ ]:
! pip install 'sagemaker[local]' --upgrade

In [ ]:
from sagemaker.local import LocalSession

# Configure our local training session
sagemaker_local_session = LocalSession()
sagemaker_local_session.config = {'local': {'local_code': True}}

In [5]:
ds_config={
  "train_batch_size": 8,
  "gradient_accumulation_steps": 1,
  "optimizer": {
    "type": "Adam",
    "params": {
      "lr": 0.00015
    }
  },
  "fp16": {
    "enabled": True
  },
  "zero_optimization": True
}

In [ ]:
est = sagemaker.estimator.Estimator(image,
                                    role=role,
                                    instance_count=1,
                                    instance_type='local_gpu',
                                    hyperparameters = ds_config,
                                    sagemaker_session=sagemaker_local_session
)

est.fit()

# Remote start

In [6]:
est = sagemaker.estimator.Estimator(image,
                                    role=role,
                                    instance_count=2,
                                    instance_type='ml.p3.16xlarge',
                                    hyperparameters = ds_config,
                                    sagemaker_session=sm_session
)

est.fit()

2021-02-11 23:37:14 Starting - Starting the training job...
2021-02-11 23:37:39 Starting - Launching requested ML instancesProfilerReport-1613086634: InProgress
.........
2021-02-11 23:39:02 Starting - Preparing the instances for training.........
2021-02-11 23:40:41 Downloading - Downloading input data
2021-02-11 23:40:41 Training - Downloading the training image....................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-02-11 23:43:57,285 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-02-11 23:43:57,352 sagemaker-training-toolkit INFO     Failed to parse hyperparameter optimizer value {'type': 'Adam', 'params': {'lr': 0.00015}} to Json.
Returning the value itself
2021-02-11 23:43:57,352 sagemaker-training-toolkit INFO     Failed to parse hyperparameter zero_optimization value True to Json.
Returning the value itself
2021-02-11 23:43:57,352 sagemaker-training-